In [ ]:
import matplotlib
matplotlib.use('Agg')
from pathlib import Path
import matplotlib.pyplot as plt
import matplotlib.colors as colors  # Added for LogNorm
import numpy as np
from stagpy.stagyydata import StagyyData
from stagpy import field as sp_field
from stagpy import error

# --- CONFIGURATION ---
field_to_plot = "T"  # Try "eta", "T", or "basalt"
dt_Gyr = 0.005         # 5 Myr per frame (faster for testing)

snap_min = 0
snap_max = 14778

fig_width = 8
fig_height = 6

# Standard geodynamic limits
field_limits = {
    "T": {"vmin": 300, "vmax": 4000},
    "basalt": {"vmin": 0.0, "vmax": 1.0},
    "eta": {"vmin": 1e18, "vmax": 1e25}  # Viscosity limits
}

field_labels = {
    "T": "Temperature",
    "eta": "Viscosity",
    "basalt": "Basalt Fraction"
}

data_path = Path("/media/aritro/f522493b-003a-404d-a839-3e0925c674b6/Aritro/StagYY/runs/venus_01/archive/")
sdat = StagyyData(data_path)

output_dir = Path(f"{field_to_plot}_time_series")
output_dir.mkdir(parents=True, exist_ok=True)

SEC_PER_GYR = 1e9 * 365.25 * 24 * 3600

# 1. MAP SNAPSHOTS TO TIME
print(f"Scanning snapshots {snap_min} to {snap_max}...")
snap_indices = []
snap_times_seconds = []

for n in range(snap_min, snap_max + 1):
    try:
        snap = sdat.snaps[n]
        t = snap.timeinfo["t"]
        snap_indices.append(snap.isnap)
        snap_times_seconds.append(t)
    except:
        continue

snap_times_seconds = np.array(snap_times_seconds)
snap_indices = np.array(snap_indices)

if len(snap_times_seconds) == 0:
    print("Error: No data found.")
else:
    t_start = snap_times_seconds.min()
    t_end = snap_times_seconds.max()
    target_times = np.arange(t_start, t_end, dt_Gyr * SEC_PER_GYR)

    print(f"Generating {len(target_times)} frames...")

    for i, t_target in enumerate(target_times):
        idx = np.abs(snap_times_seconds - t_target).argmin()
        snap_number = int(snap_indices[idx])
        
        try:
            snapshot = sdat.snaps[snap_number]
            lims = field_limits.get(field_to_plot)
            
            # --- FIXED LOGIC FOR LOGARITHMIC PLOTS (ETA) ---
            if field_to_plot == "eta":
                # For viscosity, we create a LogNorm to avoid the vmin/vmax conflict
                norm = colors.LogNorm(vmin=lims["vmin"], vmax=lims["vmax"])
                fig, ax, mesh, cbar = sp_field.plot_scalar(snapshot, field_to_plot, norm=norm)
            else:
                # For T and Basalt, standard vmin/vmax works fine
                fig, ax, mesh, cbar = sp_field.plot_scalar(snapshot, field_to_plot, 
                                                         vmin=lims["vmin"], vmax=lims["vmax"])
            
            # Labels and Text
            unit = snapshot.fields[field_to_plot].meta.dim
            display_name = field_labels.get(field_to_plot, field_to_plot)
            cbar.set_label(f"{display_name} [{unit}]")
            
            time_Gyr = t_target / SEC_PER_GYR
            ax.text(0.5, 0.5, f"{time_Gyr:.3f} Gyr", 
                    transform=ax.transAxes, ha="center", va="center", 
                    fontsize=20, color="black", 
                    bbox=dict(facecolor="white", alpha=0.7, edgecolor="none"))
            
            fig.set_size_inches(fig_width, fig_height)
            plt.tight_layout()
            
            fig.savefig(output_dir / f"frame_{i:05d}.png", dpi=400)
            plt.close(fig) # Critical to prevent the "More than 20 figures" warning
            
            if i % 50 == 0:
                print(f"Frame {i}: Snap {snap_number} ({time_Gyr:.3f} Gyr)")
                
        except Exception as e:
            print(f"Error at frame {i}: {e}")
            plt.close() # Ensure figure is closed even if it fails